In [43]:
# Import necessary libraries
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister
from qiskit_aer import Aer
from qiskit.visualization import plot_histogram
from qiskit.quantum_info import Statevector
import matplotlib.pyplot as plt
import numpy as np
import os

In [44]:
def create_circuit():
    """Create the quantum circuit with specified gates"""
    qr = QuantumRegister(2, 'q')
    cr = ClassicalRegister(2, 'c')
    qc = QuantumCircuit(qr, cr)
    
    # Apply gates in sequence
    qc.h(qr[0])
    qc.cx(qr[0], qr[1])
    qc.z(qr[1])
    qc.cx(qr[1], qr[0])
    qc.h(qr[1])
    qc.measure(qr, cr)
    
    return qc, qr, cr

In [45]:
def create_circuit_with_input(input_state):
    """Create circuit with specific input state"""
    qr = QuantumRegister(2, 'q')
    cr = ClassicalRegister(2, 'c')
    qc = QuantumCircuit(qr, cr)
    
    # Initialize input state
    if input_state == '01':
        qc.x(qr[1])
    elif input_state == '10':
        qc.x(qr[0])
    elif input_state == '11':
        qc.x(qr[0])
        qc.x(qr[1])
    
    # Apply the sequence of gates
    qc.h(qr[0])
    qc.cx(qr[0], qr[1])
    qc.z(qr[1])
    qc.cx(qr[1], qr[0])
    qc.h(qr[1])
    
    qc.measure(qr, cr)
    return qc